# Spotify Playlists concatenation

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
from yellowbrick.cluster import KElbowVisualizer
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practise
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt, numpy as np
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import AgglomerativeClustering
from matplotlib.colors import ListedColormap
from sklearn import metrics
import warnings
import sys
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from yellowbrick.cluster import SilhouetteVisualizer
import matplotlib.pyplot as plt


In [2]:
#df=pd.read_csv("Data/songs_db.csv")

In [3]:
df=pd.read_csv("Data/Denny_song_data.csv")

In [4]:
df.head()

,Unnamed: 0,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,track_name,artist_name,artist_id
0,0,0.776,0.378,0,-8.035,1,0.0322,0.435,0.001870,0.1100,...,audio_features,2EgfLUS0jNiujIWc3ZLEtn,spotify:track:2EgfLUS0jNiujIWc3ZLEtn,https://api.spotify.com/v1/tracks/2EgfLUS0jNiu...,https://api.spotify.com/v1/audio-analysis/2Egf...,175861,4,Tangerine,Tim Atlas,3CiuXDKttPUT0tWGHicFUH
1,1,0.663,0.697,0,-5.503,1,0.0508,0.272,0.008860,0.1530,...,audio_features,2ngRZDAluwYoJeuqEA4dhK,spotify:track:2ngRZDAluwYoJeuqEA4dhK,https://api.spotify.com/v1/tracks/2ngRZDAluwYo...,https://api.spotify.com/v1/audio-analysis/2ngR...,198384,4,Sidestep,Tim Atlas,3CiuXDKttPUT0tWGHicFUH
2,2,0.596,0.675,9,-7.790,1,0.0517,0.188,0.739000,0.1020,...,audio_features,3tcJ3yUXKtJpsgpAyVzP7R,spotify:track:3tcJ3yUXKtJpsgpAyVzP7R,https://api.spotify.com/v1/tracks/3tcJ3yUXKtJp...,https://api.spotify.com/v1/audio-analysis/3tcJ...,242163,4,Crime of Passion,Tim Atlas,3CiuXDKttPUT0tWGHicFUH
3,3,0.593,0.274,2,-15.402,1,0.2780,0.934,0.000569,0.0758,...,audio_features,6W4osAjSVCvUwOlVFBP76n,spotify:track:6W4osAjSVCvUwOlVFBP76n,https://api.spotify.com/v1/tracks/6W4osAjSVCvU...,https://api.spotify.com/v1/audio-analysis/6W4o...,226683,4,Together Lonely,Tim Atlas,3CiuXDKttPUT0tWGHicFUH
4,4,0.871,0.281,9,-10.650,0,0.0466,0.767,0.000180,0.1470,...,audio_features,1vdpFZ4rsQevl8WC6m3m9y,spotify:track:1vdpFZ4rsQevl8WC6m3m9y,https://api.spotify.com/v1/tracks/1vdpFZ4rsQev...,https://api.spotify.com/v1/audio-analysis/1vdp...,184259,4,Small Talk,Tim Atlas,3CiuXDKttPUT0tWGHicFUH


In [5]:
def clean_data(df):
    
    df=df.dropna()
    df=df.drop_duplicates(inplace=True)
    
    return df

In [6]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
df.rename(columns={"track_name": "song_name"},inplace=True)

In [8]:
df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,song_name,artist_name,artist_id
0,0.776,0.378,0,-8.035,1,0.0322,0.4350,0.001870,0.1100,0.453,...,audio_features,2EgfLUS0jNiujIWc3ZLEtn,spotify:track:2EgfLUS0jNiujIWc3ZLEtn,https://api.spotify.com/v1/tracks/2EgfLUS0jNiu...,https://api.spotify.com/v1/audio-analysis/2Egf...,175861,4,Tangerine,Tim Atlas,3CiuXDKttPUT0tWGHicFUH
1,0.663,0.697,0,-5.503,1,0.0508,0.2720,0.008860,0.1530,0.873,...,audio_features,2ngRZDAluwYoJeuqEA4dhK,spotify:track:2ngRZDAluwYoJeuqEA4dhK,https://api.spotify.com/v1/tracks/2ngRZDAluwYo...,https://api.spotify.com/v1/audio-analysis/2ngR...,198384,4,Sidestep,Tim Atlas,3CiuXDKttPUT0tWGHicFUH
2,0.596,0.675,9,-7.790,1,0.0517,0.1880,0.739000,0.1020,0.155,...,audio_features,3tcJ3yUXKtJpsgpAyVzP7R,spotify:track:3tcJ3yUXKtJpsgpAyVzP7R,https://api.spotify.com/v1/tracks/3tcJ3yUXKtJp...,https://api.spotify.com/v1/audio-analysis/3tcJ...,242163,4,Crime of Passion,Tim Atlas,3CiuXDKttPUT0tWGHicFUH
3,0.593,0.274,2,-15.402,1,0.2780,0.9340,0.000569,0.0758,0.346,...,audio_features,6W4osAjSVCvUwOlVFBP76n,spotify:track:6W4osAjSVCvUwOlVFBP76n,https://api.spotify.com/v1/tracks/6W4osAjSVCvU...,https://api.spotify.com/v1/audio-analysis/6W4o...,226683,4,Together Lonely,Tim Atlas,3CiuXDKttPUT0tWGHicFUH
4,0.871,0.281,9,-10.650,0,0.0466,0.7670,0.000180,0.1470,0.541,...,audio_features,1vdpFZ4rsQevl8WC6m3m9y,spotify:track:1vdpFZ4rsQevl8WC6m3m9y,https://api.spotify.com/v1/tracks/1vdpFZ4rsQev...,https://api.spotify.com/v1/audio-analysis/1vdp...,184259,4,Small Talk,Tim Atlas,3CiuXDKttPUT0tWGHicFUH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222308,0.906,0.724,6,-4.109,0,0.0931,0.4460,0.000000,0.0848,0.744,...,audio_features,4dUlJyHgdz6IeCJIYMHHDm,spotify:track:4dUlJyHgdz6IeCJIYMHHDm,https://api.spotify.com/v1/tracks/4dUlJyHgdz6I...,https://api.spotify.com/v1/audio-analysis/4dUl...,164769,4,Me Gusta - Remix (feat. Cardi B & 24kGoldn),Cardi B,4kYSro6naA4h99UJvo89HB
222309,0.934,0.443,1,-7.541,1,0.4100,0.0272,0.000000,0.0889,0.359,...,audio_features,051wt8AyLFgYnVuberd3vO,spotify:track:051wt8AyLFgYnVuberd3vO,https://api.spotify.com/v1/tracks/051wt8AyLFgY...,https://api.spotify.com/v1/audio-analysis/051w...,187541,4,WAP (feat. Megan Thee Stallion),Cardi B,4kYSro6naA4h99UJvo89HB
222310,0.903,0.447,6,-11.554,1,0.1160,0.0873,0.000000,0.1360,0.239,...,audio_features,3DyiAk1BzIF8rq9rimypG4,spotify:track:3DyiAk1BzIF8rq9rimypG4,https://api.spotify.com/v1/tracks/3DyiAk1BzIF8...,https://api.spotify.com/v1/audio-analysis/3Dyi...,374545,4,La Bebe - Remix,Cardi B,4kYSro6naA4h99UJvo89HB
222311,0.805,0.835,0,-4.603,1,0.0896,0.1300,0.000005,0.3650,0.722,...,audio_features,1EJgymgJHcjSOGSHcYaxvW,spotify:track:1EJgymgJHcjSOGSHcYaxvW,https://api.spotify.com/v1/tracks/1EJgymgJHcjS...,https://api.spotify.com/v1/audio-analysis/1EJg...,188230,4,South of the Border (feat. Camila Cabello & Ca...,Cardi B,4kYSro6naA4h99UJvo89HB


In [9]:
df.to_csv("Data/clean_song_v2_db", index=False) 

## Clustering

In [10]:
track_uri=df.uri

In [11]:
df.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,222313.000000,222313.000000,222313.000000,222313.000000,222313.000000,222313.000000,222313.000000,222313.000000,222313.000000,222313.000000,222313.000000,2.223130e+05,222313.000000
mean,0.572925,0.624311,5.277190,-8.831180,0.621183,0.098195,0.279866,0.200119,0.219634,0.448919,121.531551,2.349372e+05,3.912205
std,0.179407,0.244550,3.605864,5.031409,0.485094,0.124871,0.323469,0.333565,0.205911,0.252089,29.527075,1.347118e+05,0.443912
min,0.000000,0.000000,0.000000,-60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.312000e+03,0.000000
25%,0.454000,0.463000,2.000000,-10.614000,0.000000,0.036700,0.014000,0.000000,0.097300,0.239000,99.857000,1.794130e+05,4.000000
50%,0.588000,0.663000,5.000000,-7.623000,1.000000,0.050600,0.121000,0.000630,0.129000,0.434000,121.990000,2.180530e+05,4.000000
75%,0.708000,0.824000,9.000000,-5.565000,1.000000,0.094900,0.497000,0.283000,0.274000,0.645000,139.400000,2.685000e+05,4.000000
max,0.995000,1.000000,11.000000,2.781000,1.000000,0.965000,0.996000,1.000000,1.000000,1.000000,248.019000,5.823661e+06,5.000000


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222313 entries, 0 to 222312
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   danceability      222313 non-null  float64
 1   energy            222313 non-null  float64
 2   key               222313 non-null  int64  
 3   loudness          222313 non-null  float64
 4   mode              222313 non-null  int64  
 5   speechiness       222313 non-null  float64
 6   acousticness      222313 non-null  float64
 7   instrumentalness  222313 non-null  float64
 8   liveness          222313 non-null  float64
 9   valence           222313 non-null  float64
 10  tempo             222313 non-null  float64
 11  type              222313 non-null  object 
 12  id                222313 non-null  object 
 13  uri               222313 non-null  object 
 14  track_href        222313 non-null  object 
 15  analysis_url      222313 non-null  object 
 16  duration_ms       22

## Scaling

In [ ]:
#dropping the categorical variables for scaling
X=df._get_numeric_data()

In [ ]:
X

In [ ]:

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)
display(X.head())

In [ ]:
X_scaled_df.describe()

## Scaled dataframe

In [ ]:
X_scaled_df

## k-Means

In [ ]:
model = KMeans()
# k is range of number of clusters.
visualizer = KElbowVisualizer(model, K = range(2, 21))
visualizer.fit(X_scaled_df)        # Fit data to visualizer
visualizer.show()        # Finalize and render figure

According to this chart we need 6 clusters. 

In [ ]:
#Our optimal number of clusters is 6
kmeans = KMeans(n_clusters=6, random_state=1234)
kmeans.fit(X_scaled_df)

In [ ]:
len(kmeans.labels_)


In [ ]:
clusters = kmeans.predict(X_scaled_df)
#clusters
pd.Series(clusters).value_counts().sort_index()

In [ ]:
#X_df = pd.DataFrame(X)
X["cluster"] = clusters
X.head()

In [ ]:
# assign a cluster to each example
labels = kmeans.predict(X_scaled_df)
# retrieve unique clusters
clusters = np.unique(labels)
# create scatter plot for samples from each cluster
for cluster in clusters:
    # get row indexes for samples with this cluster
    row_ix = np.where(labels == cluster)
    # create scatter of these samples
    pyplot.scatter(X.to_numpy()[row_ix, 0], X.to_numpy()[row_ix, 1])
    # show the plot
pyplot.show()

In [ ]:
categorical_df=df.select_dtypes('object')

In [ ]:
df=pd.concat([X, categorical_df], axis=1)

In [ ]:
# analyzing our results, we can start to see some winners
df.groupby(['cluster', 'artist_name'], as_index=False).count().sort_values(['cluster', 'track_name'], ascending=False)[['cluster', 'artist_name', 'track_name']]

In [ ]:
df.to_csv("Data/clustered_df_v2.csv", index=False) 

In [ ]:
df

# Saving with pickle

In [ ]:
import pickle

#scaler = StandardScaler()
#model = KMeans()

with open("Model/scaler.pickle", "wb") as f:
    pickle.dump(scaler,f)

with open("Model/kmeans_4.pickle", "wb") as f:
    pickle.dump(kmeans,f)